In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pynapple as nap
from spatial_manifolds.toroidal import *
from spatial_manifolds.behaviour_plots import *
from matplotlib.colors import TwoSlopeNorm
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
import umap
from cebra import CEBRA
import cebra.integrations.plotly
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from spatial_manifolds.mlencoding import *
from spatial_manifolds.circular_decoder import circular_decoder, cross_validate_decoder, cross_validate_decoder_time, circular_nanmean
from spatial_manifolds.data.curation import curate_clusters
from scipy.stats import zscore
from spatial_manifolds.util import gaussian_filter_nan
from spatial_manifolds.predictive_grid import compute_travel_projected, wrap_list
from spatial_manifolds.behaviour_plots import *
from spatial_manifolds.behaviour_plots import trial_cat_priority
from spatial_manifolds.detect_grids import *
from spatial_manifolds.brainrender_helper import *

import numpy as np
import matplotlib.pyplot as plt
import hdbscan
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
fig_path = '/Users/harryclark/Documents/figs/FIGURE1/'
mouse = 29
day = 23

# good examples include 
#mice = [25, 25, 26, 27, 29, 28]
#days = [25, 24, 18, 26, 23, 25]

In [ ]:
gcs, ngs, ns, sc, ngs_ns, all = cell_classification_of1(mouse, day, percentile_threshold=99) # subset
rc, rsc, vr_ns = cell_classification_vr(mouse, day)

g_m_ids, g_m_cluster_ids = HDBSCAN_grid_modules(gcs, all, mouse, day, min_cluster_size=3, cluster_selection_epsilon=3, 
                                                figpath=fig_path, curate_with_vr=True, curate_with_brain_region=True) # create grid modules using HDBSCAN    

plot_grid_modules_rate_maps(gcs, g_m_ids, g_m_cluster_ids, mouse, day, figpath=fig_path)

# we now have cluster ids classified into modules, non grid spatial cells and non spatial cells 
# as defined by activity in the open field
g_m_cluster_ids = sorted(g_m_cluster_ids, key=len, reverse=True) 
cluster_ids_by_group = []
cluster_ids_by_group.extend(g_m_cluster_ids) # grid cells by module [0,1,2...]
cluster_ids_by_group.append(ngs.cluster_id.values.tolist()) # non grid spatial [-4]
cluster_ids_by_group.append(ns.cluster_id.values.tolist()) # non spatial cells [-3]
cluster_ids_by_group.append(gcs.cluster_id.values.tolist()) # all grid cells [-2]
cluster_ids_by_group.append(sc.cluster_id.values.tolist()) # speed cells [-1]

for m, cluster_ids in enumerate(cluster_ids_by_group):
    plot_vr_rate_maps(mouse, day, cluster_ids, label=f'{m}', figpath=fig_path)

#plot_vr_rate_maps(mouse, day, rc.cluster_id.values, label=f'ramp_cells', figpath=fig_path)
#plot_vr_rate_maps(mouse, day, rsc.cluster_id.values, label=f'speed_ramp_cells', figpath=fig_path)

In [ ]:
cluster_ids=cluster_ids_by_group[0]
tcs,tcs_time,_,_,_,_ = compute_vr_tcs(mouse, day)
ncols = 10
nrows = int(np.ceil(len(cluster_ids)/ncols))
fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(10, 1.4*nrows), squeeze=False)
counter = 0
for j in range(nrows):
    for i in range(ncols):
        if counter<len(cluster_ids):
            index = cluster_ids[counter]
            plot_firing_rate_map(ax[j, i], 
                                zscore(tcs[index]),
                                bs=bs,
                                tl=tl,
                                p=95)
        else:
            ax[j, i].axis('off')
        counter+=1
        ax[j, i].set_xticks([])
        ax[j, i].set_yticks([])
        ax[j, i].xaxis.set_tick_params(labelbottom=False)
        ax[j, i].yaxis.set_tick_params(labelleft=False)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.signal import welch

for id in ngs.cluster_id:
    cluster = all[all.cluster_id == id]
    print(f'{cluster.cluster_id.iloc[0]}, x={np.round(cluster.SC_x.iloc[0])}, y={np.round(cluster.SC_y.iloc[0])}, probe_x={np.round(cluster.probe_x.iloc[0])}, probe_y={np.round(cluster.probe_y.iloc[0])}')
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(1.5,1.5), squeeze=False)
    plot_firing_rate_map(ax[0,0], zscore(tcs[id]), bs=bs, tl=tl, p=95)
    plt.show()
'''
    frequencies, psd = welch(tcs_time[id], fs=int(1000/time_bs))
    fig, ax = plt.subplots(figsize=(1.5, 1.5))
    ax.plot(frequencies[:50], psd[:50]*frequencies[:50])
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('PSD')
    plt.show()
'''


In [ ]:
from scipy.signal import welch

for id in gcs.cluster_id:
    cluster = all[all.cluster_id == id]
    print(f'{cluster.cluster_id.iloc[0]}, x={np.round(cluster.SC_x.iloc[0])}, y={np.round(cluster.SC_y.iloc[0])}, probe_x={np.round(cluster.probe_x.iloc[0])}, probe_y={np.round(cluster.probe_y.iloc[0])}')
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(1.5,1.5), squeeze=False)
    plot_firing_rate_map(ax[0,0], zscore(tcs[id]), bs=bs, tl=tl, p=95)
    plt.show()
'''
    frequencies, psd = welch(tcs_time[id], fs=int(1000/time_bs))
    fig, ax = plt.subplots(figsize=(1.5, 1.5))
    ax.plot(frequencies[:50], psd[:50]*frequencies[:50])
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('PSD')
    plt.show()
'''


In [ ]:
len(gcs)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2, 8), squeeze=False)
plot_NP2_probe(ax[0,0], sorting_analyzer_path='/Users/harryclark/Downloads/kilosort4_sa')
ax[0,0].scatter(ngs['probe_x'], ngs['probe_y'], color='green', alpha=0.3)
ax[0,0].scatter(gcs['probe_x'], gcs['probe_y'], color='blue', alpha=0.3)
plt.show()

pick 2 interesting grid cells to use 

In [ ]:
reference_grid_cell_id1 = 194
reference_grid_cell_id2 = 160
reference_non_grid_cell_id3 = 411

In [ ]:
tcs, tcs_time, _, last_ephys_bin, beh, clusters = compute_vr_tcs(mouse,day, apply_zscore=False, apply_guassian_filter=False)

last_ephys_time_bin = clusters[clusters.index[0]].count(bin_size=time_bs, time_units = 'ms').index[-1]

# time binned variables for later
ep = nap.IntervalSet(start=0, end=last_ephys_time_bin, time_units = 's')
speed_in_time = np.array(beh['S'].bin_average(bin_size=time_bs, time_units = 'ms', ep=ep))
dt_in_time = np.array(beh['travel'].bin_average(bin_size=time_bs, time_units = 'ms', ep=ep)-((beh['trial_number'][0]-1)*tl))
pos_in_time = dt_in_time%tl
trial_number_in_time = (dt_in_time//tl)+beh['trial_number'][0]

if np.any(np.isnan(pos_in_time)):
    series = pd.Series(dt_in_time)
    filled_series = series.ffill().bfill()
    dt_in_time = np.array(filled_series)
    pos_in_time = dt_in_time%tl
    trial_number_in_time = (dt_in_time//tl)+beh['trial_number'][0]

if np.any(np.isnan(speed_in_time)):
    series = pd.Series(speed_in_time)
    filled_series = series.ffill().bfill()
    speed_in_time = np.array(speed_in_time)

were going to use a history of 2000ms

were going to test to see if theres a relationship between the pseudo r2 value when you train on a grid cell to predict the firing of another grid cell and vice versa. 

In [ ]:
#reference_grid_cell_id1
ref = np.array(tcs_time[reference_grid_cell_id1])

vals1=[]
vals2=[]
for gc_id in gcs.cluster_id:
    alt = np.array(tcs_time[gc_id])
    if gc_id != reference_grid_cell_id1:
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 2000)
        Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 2, continuous_folds = True)
        vals1.append(np.nanmean(pR2_cv))
        Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 2, continuous_folds = True)
        vals2.append(np.nanmean(pR2_cv))
plt.scatter(vals1,vals2, color='blue', alpha=0.3)

vals1=[]
vals2=[]
for ngs_id in ngs.cluster_id:
    alt = np.array(tcs_time[ngs_id])   
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 2000)
    Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 2, continuous_folds = True)
    vals1.append(np.nanmean(pR2_cv))
    Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 2, continuous_folds = True)
    vals2.append(np.nanmean(pR2_cv))
plt.scatter(vals1,vals2, color='green', alpha=0.3)

In [ ]:
plt.scatter(vals1,vals2, color='blue')
plt.xscale('log')
plt.yscale('log')

In [ ]:
#reference_grid_cell_id1
ref = np.array(tcs_time[reference_grid_cell_id1])

gc_vals1=[]
gc_vals2=[]
for i, gc_id in enumerate(gcs.cluster_id):
    alt = np.array(tcs_time[gc_id])
    if gc_id != reference_grid_cell_id1:
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 50)
        Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 0, continuous_folds = True)
        gc_vals1.append(np.nanmean(pR2_cv))
        Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 0, continuous_folds = True)
        gc_vals2.append(np.nanmean(pR2_cv))
        print(f'done {i+1}/{len(gcs)}')

ngs_vals1=[]
ngs_vals2=[]
for i, ngs_id in enumerate(ngs.cluster_id):
    alt = np.array(tcs_time[ngs_id])   
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 50)
    Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 0, continuous_folds = True)
    ngs_vals1.append(np.nanmean(pR2_cv))
    Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 0, continuous_folds = True)
    ngs_vals2.append(np.nanmean(pR2_cv))
    print(f'done {i+1}/{len(ngs)}')


shuf_vals1=[]
shuf_vals2=[]
for i, gc_id in enumerate(gcs.cluster_id):
    alt = np.array(tcs_time[gc_id])
    np.random.shuffle(alt)
    if gc_id != reference_grid_cell_id1:
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 50)
        Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 0, continuous_folds = True)
        shuf_vals1.append(np.nanmean(pR2_cv))
        Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 0, continuous_folds = True)
        shuf_vals2.append(np.nanmean(pR2_cv))
        print(f'done {i+1}/{len(gcs)}')

plt.scatter(shuf_vals1, shuf_vals2, color='grey', alpha=0.3)
plt.axvline(np.nanpercentile(shuf_vals1,95), color='grey', alpha=0.3)
plt.axvline(np.nanpercentile(shuf_vals1,99), color='grey', alpha=0.7)
plt.axhline(np.nanpercentile(shuf_vals2,95), color='grey', alpha=0.3)
plt.axhline(np.nanpercentile(shuf_vals2,99), color='grey', alpha=0.7)
plt.scatter(ngs_vals1, ngs_vals2, color='green', alpha=0.3)
plt.scatter(gc_vals1,  gc_vals2, color='blue', alpha=0.3)
plt.xscale('log')
plt.yscale('log')


In [ ]:

plt.scatter(shuf_vals1, shuf_vals2, color='grey', alpha=0.3)
plt.axvline(np.nanpercentile(shuf_vals1,95), color='grey', alpha=0.3)
plt.axvline(np.nanpercentile(shuf_vals1,99), color='grey', alpha=0.7)
plt.axhline(np.nanpercentile(shuf_vals2,95), color='grey', alpha=0.3)
plt.axhline(np.nanpercentile(shuf_vals2,99), color='grey', alpha=0.7)
plt.scatter(ngs_vals1, ngs_vals2, color='green', alpha=0.3)
plt.scatter(gc_vals1,  gc_vals2, color='blue', alpha=0.3)



In [ ]:
#reference_grid_cell_id1
ref = np.array(tcs_time[reference_grid_cell_id1])

lh_gc_vals1=[]
lh_gc_vals2=[]
lh_gc_vals3=[]
for i, gc_id in enumerate(gcs.cluster_id):
    alt = np.array(tcs_time[gc_id])
    if gc_id != reference_grid_cell_id1:
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 2000)
        Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 0, continuous_folds = True)
        lh_gc_vals1.append(np.nanmean(pR2_cv))
        Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 0, continuous_folds = True)
        lh_gc_vals2.append(np.nanmean(pR2_cv))
        dist = distance.euclidean([all[all['cluster_id'] == reference_grid_cell_id1].probe_x.iloc[0],  
                                   all[all['cluster_id'] == reference_grid_cell_id1].probe_y.iloc[0]],
                                   [all[all['cluster_id'] == gc_id].probe_x.iloc[0],  
                                   all[all['cluster_id'] == gc_id].probe_y.iloc[0]])
        lh_gc_vals3.append(dist)
        print(f'done {i+1}/{len(gcs)}')

lh_ngs_vals1=[]
lh_ngs_vals2=[]
lh_ngs_vals3=[]
for i, ngs_id in enumerate(ngs.cluster_id):
    alt = np.array(tcs_time[ngs_id])   
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 2000)
    Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 0, continuous_folds = True)
    lh_ngs_vals1.append(np.nanmean(pR2_cv))
    Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 0, continuous_folds = True)
    lh_ngs_vals2.append(np.nanmean(pR2_cv))
    dist = distance.euclidean([all[all['cluster_id'] == reference_grid_cell_id1].probe_x.iloc[0],  
                                   all[all['cluster_id'] == reference_grid_cell_id1].probe_y.iloc[0]],
                                   [all[all['cluster_id'] == ngs_id].probe_x.iloc[0],  
                                   all[all['cluster_id'] == ngs_id].probe_y.iloc[0]])
    lh_ngs_vals3.append(dist)
    print(f'done {i+1}/{len(ngs)}')


lh_shuf_vals1=[]
lh_shuf_vals2=[]
for i, gc_id in enumerate(gcs.cluster_id):
    alt = np.array(tcs_time[gc_id])
    np.random.shuffle(alt)
    if gc_id != reference_grid_cell_id1:
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False,
                         window = time_bs, n_filters = 5, max_time = 50)
        Y_hat, pR2_cv = xgb_history.fit_cv(alt, ref, verbose = 0, continuous_folds = True)
        lh_shuf_vals1.append(np.nanmean(pR2_cv))
        Y_hat, pR2_cv = xgb_history.fit_cv(ref, alt, verbose = 0, continuous_folds = True)
        lh_shuf_vals2.append(np.nanmean(pR2_cv))
        print(f'done {i+1}/{len(gcs)}')

In [ ]:

plt.scatter(lh_shuf_vals1, lh_shuf_vals2, color='grey', alpha=0.3)
plt.axvline(np.nanpercentile(lh_shuf_vals1,95), color='grey', alpha=0.3)
plt.axvline(np.nanpercentile(lh_shuf_vals1,99), color='grey', alpha=0.7)
plt.axhline(np.nanpercentile(lh_shuf_vals2,95), color='grey', alpha=0.3)
plt.axhline(np.nanpercentile(lh_shuf_vals2,99), color='grey', alpha=0.7)
plt.scatter(lh_ngs_vals1, lh_ngs_vals2, color='green', alpha=0.3)
plt.scatter(lh_gc_vals1,  lh_gc_vals2, color='blue', alpha=0.3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('pR2 - predict ref GC with cell')
plt.ylabel('pR2 - predict cell with ref GC')



In [ ]:
plt.scatter(lh_ngs_vals1, lh_ngs_vals3, color='green', alpha=0.3)
plt.scatter(lh_gc_vals1,  lh_gc_vals3, color='blue', alpha=0.3)
plt.xscale('log')


In [ ]:
all

In [ ]:
gc_id1 = cluster_ids_by_group[0][9]
gc_id2 = cluster_ids_by_group[0][10]
distal_id = 316

y1 = np.array(tcs_time[gc_id1])
y2 = np.array(tcs_time[gc_id2])
y3 = np.array(tcs_time[distal_id])

X  = np.stack([pos_in_time, speed_in_time]).T
X1 = np.stack([pos_in_time, speed_in_time, y1]).T
X2 = np.stack([pos_in_time, speed_in_time, y2]).T

In [ ]:
time_bs

In [ ]:
xgb_history = MLencoding(tunemodel = 'xgboost',
                         cov_history = True, spike_history=True, # We can choose!
                         window = time_bs, #this dataset has 100ms time bins
                         n_filters = 5,
                         max_time = 1000)

Y_hat, pR2_cv = xgb_history.fit_cv(X, y1, verbose = 2, continuous_folds = True)

plot to see if the true time binned spikes aligns with the prediction of the model

In [ ]:
plt.scatter(y1, Y_hat, alpha=0.1)
plt.show()

plot the rate maps for true space binned, true time binned and predicted time binned data

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2, 2), squeeze=False)
plot_firing_rate_map(ax[0,0], zscore(tcs[gc_id1]), bs=bs, tl=tl, p=95)
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2, 2), squeeze=False)
plot_firing_rate_map(ax[0,0], zscore(tcs[gc_id2]), bs=bs, tl=tl, p=95)
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2, 2), squeeze=False)
plot_firing_rate_map(ax[0,0], zscore(tcs[distal_id]), bs=bs, tl=tl, p=95)
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 3), squeeze=False)
plot_firing_rate_map(ax[0,0], zscore(tcs[16]), bs=bs, tl=tl, p=95)
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(4, 2), squeeze=False)
tc, _ = np.histogram(dt_in_time, weights=y1, bins=last_ephys_bin, range=(0, bs*last_ephys_bin))
plot_firing_rate_map(ax[0,0], zscore(tc), bs=bs, tl=tl, p=95)
tc, _ = np.histogram(dt_in_time, weights=Y_hat, bins=last_ephys_bin, range=(0, bs*last_ephys_bin))
plot_firing_rate_map(ax[0,1], zscore(tc), bs=bs, tl=tl, p=95)
ax[0,0].set_title(f'actual')
ax[0,1].set_title(f'predicted')
plt.tight_layout()
plt.show()

we want to evaluate the effect of the history parameter. Here we will increase the number of time bins available to the sample
and plot this as a function of model performance

In [ ]:
nbins_history = np.append(np.append(np.array([5,6,7,8,9,10]), np.arange(10,100,20)), np.arange(100,501, 100))


In [ ]:
pR2s = []
for nbins in nbins_history:
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=True, 
                             window = time_bs, n_filters = 5, max_time = int(time_bs*nbins))
    Y_hat, pR2_cv = xgb_history.fit_cv(X, y1, verbose = 0, continuous_folds = True)
    pR2s.append(np.nanmean(pR2_cv))
plt.plot(nbins_history, pR2s)
plt.xlabel('history length (bins)')
plt.ylabel('pseudo R2')
plt.show()

set the coviarate history to false and the spike history to true. This is only testing prediction using the cells own history

In [ ]:
pR2s = []
for nbins in nbins_history:
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = False, spike_history=True, 
                             window = time_bs, n_filters = 5, max_time = int(time_bs*nbins))
    Y_hat, pR2_cv = xgb_history.fit_cv(X, y1, verbose = 0, continuous_folds = True)
    pR2s.append(np.nanmean(pR2_cv))
plt.plot(nbins_history, pR2s)
plt.xlabel('history length (bins)')
plt.ylabel('pseudo R2')
plt.show()

set the spike history to false and the cov history to true, this is now only using spike data from the other cell

In [ ]:
pR2s = []
for nbins in nbins_history:
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                             window = time_bs, n_filters = 5, max_time = int(time_bs*nbins))
    Y_hat, pR2_cv = xgb_history.fit_cv(y2, y1, verbose = 0, continuous_folds = True)
    pR2s.append(np.nanmean(pR2_cv))
plt.plot(nbins_history, pR2s)
plt.xlabel('history length (bins)')
plt.ylabel('pseudo R2')
plt.show()

set the spike history to false and the cov history to true, We will now look at the effect of using covariate history from a cell with a very distal connection (such as in the cerebellum)

In [ ]:
pR2s = []
for nbins in nbins_history:
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                             window = time_bs, n_filters = 5, max_time = int(time_bs*nbins))
    Y_hat, pR2_cv = xgb_history.fit_cv(y3, y1, verbose = 0, continuous_folds = True)
    pR2s.append(np.nanmean(pR2_cv))
plt.plot(nbins_history, pR2s)
plt.xlabel('history length (bins)')
plt.ylabel('pseudo R2')
plt.show()

It looks like for a number of different runs, providing a spike history of 40 time bins is optimal for model performance.

Now we're going to test model performance by dropping out n numbers of grid cells from a covariate history and see how the number of comodular grid cells improves the prediction of a held out grid cell.

In [ ]:
test_grid_cell = cluster_ids_by_group[0][9]
grid_module_population_cluster_id_minus_test_grid_cells = cluster_ids_by_group[0] 
grid_module_population_cluster_id_minus_test_grid_cells.remove(test_grid_cell)
x_grid_module = {cluster_id: tcs_time[cluster_id] for cluster_id in grid_module_population_cluster_id_minus_test_grid_cells if cluster_id in tcs_time}

all_x = np.vstack(list(x_grid_module.values())).T
y = np.array(tcs_time[test_grid_cell])

In [ ]:
n_neurons = np.arange(1, all_x.shape[1]+1, 2)
iterations = 3
ntime_bins = 5
nfilters = 5
pR2s = np.zeros((len(n_neurons), iterations))
pR2s_shuff = np.zeros((len(n_neurons), iterations))

for n in n_neurons:
    print(f'I am going to use only {n} grid cells')
    reps = []
    for i in range(iterations):
        # real data
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                             window = time_bs, n_filters = nfilters, max_time = 2000)
        np.random.seed(i)
        idx = np.random.choice(n_neurons-1, n, replace=False)
        x = all_x[:, idx]
        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2s[n-1, i] = np.nanmean(pR2_cv)
        print(f'pR2_cv = {np.nanmean(pR2_cv)}')

        # shuffle data
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                             window = time_bs, n_filters = nfilters, max_time = 2000)
        nx = x.T.copy()
        for r, xROW in enumerate(x.T):
            np.random.shuffle(xROW)
            nx[r]=xROW
        x = nx.T

        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2s_shuff[n-1, i] = np.nanmean(pR2_cv)
        print(f'pR2_cv_shuff = {np.nanmean(pR2_cv)}')
      

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2,2))

ax.plot(n_neurons, np.nanmean(pR2s, axis=1), color='red', label='real')
ax.fill_between(n_neurons, 
         np.nanmean(pR2s, axis=1)+stats.sem(pR2s, axis=1, nan_policy='omit'),
         np.nanmean(pR2s, axis=1)-stats.sem(pR2s, axis=1, nan_policy='omit'),color='red', alpha=0.3)
ax.plot(n_neurons, np.nanmean(pR2s_shuff, axis=1), color='grey', label='shuffle')
ax.fill_between(n_neurons, 
         np.nanmean(pR2s_shuff, axis=1)+stats.sem(pR2s_shuff, axis=1, nan_policy='omit'),
         np.nanmean(pR2s_shuff, axis=1)-stats.sem(pR2s_shuff, axis=1, nan_policy='omit'),color='grey', alpha=0.3)
ax.set_ylabel(f'pseudo R2')
ax.set_xlabel(f'n neurons')
ax.legend()
plt.show()

we can clear see that any number of grid cells can beat the shuffle, but there is slowing of the plateau at around 5 grid cells

what happens if we try to predict the activity of a cell with the history of another cell, and do this for all combinations within a session?
We can sorted the clusters by ML position in this session because we are recording across all the shanks. 

First we will only use 50ms of history (5 time bins of 10 ms each). 

Then we will use 2 seconds of history (200 time bins of 10 ms each).

We'll take an example grid cell from above and test the predictabliltity against all other cells first before running every combination

In [ ]:
y = np.array(tcs_time[gc_id2])[:5000]

all = all.sort_values(by=['SC_x', 'SC_y'], ascending=[False, False])

nfilters = 5
pR2s_short_history = np.zeros(len(all))
pR2s_long_history = np.zeros(len(all))

done = 0
n_to_finish = len(all)

for i, id_i in enumerate(all.cluster_id):
    x = np.array(tcs_time[id_i])[:5000]

    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                                window = time_bs, n_filters = nfilters, max_time = 50)
    Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
    pR2s_short_history[i] = np.nanmean(pR2_cv)
    
    xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                                window = time_bs, n_filters = nfilters, max_time = 2000)
    Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
    pR2s_long_history[i] = np.nanmean(pR2_cv)
    
    done+= 1
    print(f'completed {done}/{n_to_finish}')

plt.plot(pR2s_short_history, label='50ms history')
plt.plot(pR2s_long_history, label='2000ms history')
plt.legend()
plt.show()    

In [ ]:
plt.scatter(all.SC_x, all.SC_y)

In [ ]:
all = all.sort_values(by=['SC_x', 'SC_y'], ascending=[False, False])

cap = True
nfilters = 5
pR2s = np.zeros((len(all), 4))

done = 0
n_to_finish = len(all)*4

for i, id_i in enumerate(all.cluster_id):
    for j, (left, right) in enumerate(zip([-3800, -3500, -3300, -3100], [-3600, -3400,-3100, -2900])):
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                                 window = time_bs, n_filters = nfilters, max_time = 50)
        y = np.array(tcs_time[id_i])

        valid_cluster_ids = all[(all['SC_x'] > left) & 
                                (all['SC_x'] < right) & 
                                (all['cluster_id'] != id_i)]
        tcs_to_use = {cluster_id: tcs_time[cluster_id] for cluster_id in valid_cluster_ids.cluster_id.values if cluster_id in tcs_time}
        x = np.vstack(list(tcs_to_use.values())).T
        
        if cap:
            x = x[:5000,:]
            y = y[:5000]

        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2s[i, j] = np.nanmean(pR2_cv)

        done+= 1
        print(f'completed {done}/{n_to_finish}')


In [ ]:
all = all.sort_values(by=['SC_x', 'SC_y'], ascending=[False, False])

cap = True
nfilters = 5
pR2slh = np.zeros((len(all), 4))

done = 0
n_to_finish = len(all)*4

for i, id_i in enumerate(all.cluster_id):
    for j, (left, right) in enumerate(zip([-3800, -3500, -3300, -3100], [-3600, -3400,-3100, -2900])):
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                                 window = time_bs, n_filters = nfilters, max_time = 2000)
        y = np.array(tcs_time[id_i])

        valid_cluster_ids = all[(all['SC_x'] > left) & 
                                (all['SC_x'] < right) & 
                                (all['cluster_id'] != id_i)]
        tcs_to_use = {cluster_id: tcs_time[cluster_id] for cluster_id in valid_cluster_ids.cluster_id.values if cluster_id in tcs_time}
        x = np.vstack(list(tcs_to_use.values())).T
        
        if cap:
            x = x[:5000,:]
            y = y[:5000]

        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2slh[i, j] = np.nanmean(pR2_cv)

        done+= 1
        print(f'completed {done}/{n_to_finish}')


In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(3,2),sharey=True)

x = np.arange(1, len(pR2slh)+1)
y = np.arange(0, len(pR2slh[0]))
X, Y = np.meshgrid(x, y)
heatmap = ax[0].pcolormesh(Y, X, pR2s.T, shading='auto', cmap='viridis', vmin=0, vmax=0.4)
heatmap.set_rasterized(True)
ax[0].set_ylabel(f'Tested on neuron')
ax[0].set_xlabel(f'Trained on shank')
ax[0].set_xticks([0,1,2,3])
ax[0].set_xticklabels([1,2,3,4])
ax[0].invert_yaxis()
ax[0].set_title('50ms cov history')

heatmap = ax[1].pcolormesh(Y, X, pR2slh.T, shading='auto', cmap='viridis', vmin=0,vmax=0.4)
heatmap.set_rasterized(True)
ax[1].set_xlabel(f'Trained on shank')
ax[1].set_xticks([0,1,2,3])
ax[1].set_xticklabels([1,2,3,4])
ax[1].set_title('2s cov history')
plt.show()   

In [ ]:
all = all.sort_values(by=['SC_x', 'SC_y'], ascending=[False, False])

cap = True
nfilters = 5
pR2s_spike_and_covariate_history = np.zeros((len(all), 4))

done = 0
n_to_finish = len(all)*4

for i, id_i in enumerate(all.cluster_id):
    for j, (left, right) in enumerate(zip([-3800, -3500, -3300, -3100], [-3600, -3400,-3100, -2900])):
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=True, 
                                 window = time_bs, n_filters = nfilters, max_time = 50)
        y = np.array(tcs_time[id_i])

        valid_cluster_ids = all[(all['SC_x'] > left) & 
                                (all['SC_x'] < right) & 
                                (all['cluster_id'] != id_i)]
        tcs_to_use = {cluster_id: tcs_time[cluster_id] for cluster_id in valid_cluster_ids.cluster_id.values if cluster_id in tcs_time}
        x = np.vstack(list(tcs_to_use.values())).T
        
        if cap:
            x = x[:5000, :]
            y = y[:5000]

        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2s_spike_and_covariate_history[i, j] = np.nanmean(pR2_cv)
        done+= 1
        print(f'completed {done}/{n_to_finish}')


In [ ]:
all = all.sort_values(by=['SC_x', 'SC_y'], ascending=[False, False])

cap = True
nfilters = 5
pR2s_spike_history = np.zeros((len(all), 4))

done = 0
n_to_finish = len(all)*4

for i, id_i in enumerate(all.cluster_id):
    for j, (left, right) in enumerate(zip([-3800, -3500, -3300, -3100], [-3600, -3400,-3100, -2900])):
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = False, spike_history=True, 
                                 window = time_bs, n_filters = nfilters, max_time = 50)
        y = np.array(tcs_time[id_i])

        valid_cluster_ids = all[(all['SC_x'] > left) & 
                                (all['SC_x'] < right) & 
                                (all['cluster_id'] != id_i)]
        tcs_to_use = {cluster_id: tcs_time[cluster_id] for cluster_id in valid_cluster_ids.cluster_id.values if cluster_id in tcs_time}
        x = np.vstack(list(tcs_to_use.values())).T
        
        if cap:
            x = x[:5000,:]
            y = y[:5000]

        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2s_spike_history[i, j] = np.nanmean(pR2_cv)
        done+= 1
        print(f'completed {done}/{n_to_finish}')


In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(5,2),sharey=True)

x = np.arange(1, len(pR2slh)+1)
y = np.arange(0, len(pR2slh[0]))
X, Y = np.meshgrid(x, y)
heatmap = ax[0].pcolormesh(Y, X, pR2s_spike_history.T, shading='auto', cmap='viridis', vmin=0, vmax=0.3)
heatmap.set_rasterized(True)
ax[0].set_ylabel(f'Tested on neuron')
ax[0].set_xlabel(f'Trained on S')
ax[0].set_xticks([0,1,2,3])
ax[0].set_xticklabels([1,2,3,4])
ax[0].invert_yaxis()
ax[0].set_title('sp history')

heatmap = ax[1].pcolormesh(Y, X, pR2s_spike_and_covariate_history.T, shading='auto', cmap='viridis', vmin=0, vmax=0.3)
heatmap.set_rasterized(True)
ax[1].set_xlabel(f'Trained on S')
ax[1].set_xticks([0,1,2,3])
ax[1].set_xticklabels([1,2,3,4])
ax[1].set_title('sp&cov history')

heatmap = ax[2].pcolormesh(Y, X, pR2s.T, shading='auto', cmap='viridis', vmin=0, vmax=0.3)
heatmap.set_rasterized(True)
ax[2].set_xlabel(f'Trained on S')
ax[2].set_xticks([0,1,2,3])
ax[2].set_xticklabels([1,2,3,4])
ax[2].set_title('cov history')


heatmap = ax[3].pcolormesh(Y, X, pR2s_spike_and_covariate_history.T-pR2s_spike_history.T, shading='auto', cmap='coolwarm', vmin=-0.1, vmax=0.1)
heatmap.set_rasterized(True)
ax[3].set_xlabel(f'Trained on S')
ax[3].set_xticks([0,1,2,3])
ax[3].set_xticklabels([1,2,3,4])
ax[3].set_title('spcov minus sp')

plt.show()   

In [ ]:
all = all.sort_values(by=['SC_x'], ascending=[False])


In [ ]:
plt.imshow(pR2s, cmap='viridis', vmin=0, vmax=0.2)
plt.colorbar()
plt.show()    

In [ ]:
all = all.sort_values(by=['SC_x', 'SC_y'], ascending=[False, False])

nfilters = 5
pR2s = np.zeros((len(all), len(all)))

done = 0
n_to_finish = len(all)*len(all)

for i, id_i in enumerate(all.cluster_id):
    for j, id_j in enumerate(all.cluster_id):
        xgb_history = MLencoding(tunemodel = 'xgboost', cov_history = True, spike_history=False, 
                                 window = time_bs, n_filters = nfilters, max_time = 2000)
        x = np.array(tcs_time[id_i][:50000])
        y = np.array(tcs_time[id_j][:50000])
        Y_hat, pR2_cv = xgb_history.fit_cv(x, y, verbose = 0, continuous_folds = True)
        pR2s[i, j] = np.nanmean(pR2_cv)

        done+= 1
        print(f'completed {done}/{n_to_finish}')

plt.imshow(pR2s, cmap='viridis')
plt.colorbar()
plt.show()    

In [ ]:
from hyperopt import fmin, hp, Trials, tpe, STATUS_OK

# Makes sure these are in model.params, otherwise you'll get a key error
space4rf = {
    'silent': 1,
    'learning_rate': hp.choice('learning_rate', [0.01, 0.05, 0.1, 0.2, 0.3]),
    'min_child_weight': hp.choice('min_child_weight', [1, 2, 5, 10]),
    'n_estimators': hp.choice('n_estimators', [100, 300, 500, 700, 1000]),
    'subsample': hp.choice('subsample', [0.5, 0.6, 0.7, 0.8, 1.0]),
    'max_depth': hp.choice('max_depth', [3, 5, 7, 9]),
    'gamma': hp.choice('gamma', [0, 0.1, 0.3, 0.5, 1.0]),
}

#object that holds iteration results
trials = Trials()

#define model
xgb_history = MLencoding(tunemodel = 'xgboost',
                         cov_history = False, spike_history=True, # We can choose!
                         window = time_bs, #this dataset has 50ms time bins
                         n_filters = 5,
                         max_time = int(time_bs*10))

#function to minimize
def fnc(params):
    print(params)

    # make sure parameters are integers that need to be. 
    params['silent'] = int(params['silent'])
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_child_weight'] = int(params['min_child_weight'])

    xgb_history.set_params(params)
    
    # Remember that X and y have been defined above.
    Y_hat, PR2s = xgb_history.fit_cv(X,y, n_cv = 5, verbose = 0, continuous_folds = True)

    # return negative since hyperopt always minimizes the function
    return -np.mean(PR2s)

hyperoptBest = fmin(fnc, space4rf, algo=tpe.suggest, max_evals=500, 
                    trials=trials, return_argmin=False)


In [ ]:
hyperoptBest